In [4]:
from usagrid import s3

In [5]:
from dataclasses import dataclass,field
from typing import List

@dataclass
class Setup:
    YEAR:str= "2022"
    Expected_Start_Date:str= "2022-01-01"
    Expected_End_Date:str= "2022-12-31"
    PARSE_DATES:List[str]=field(default_factory=lambda: ["GMT MKT Interval"])

In [6]:
@dataclass
class generation_mix_historical:
    COAL_MARKET:str= "float64"
    COAL_SELF:str= "float64"
    DIESEL_FUEL_OIL_MARKET:str= "float64"
    DIESEL_FUEL_OIL_SELF:str= "float64"
    HYDRO_MARKET:str= "float64"
    HYDRO_SELF:str= "float64"
    NATURAL_GAS_MARKET:str= "float64"
    GAS_SELF:str= "float64"
    NUCLEAR_MARKET:str= "float64"
    NUCLEAR_SELF:str= "float64"
    SOLAR_MARKET:str= "float64"
    SOLAR_SELF:str= "float64"
    WASTE_DISPOSAL_SERVICES_MARKET:str= "float64"
    WASTE_DISPOSAL_SERVICES_SELF:str= "float64"
    WIND_MARKET:str= "float64"
    WIND_SELF:str= "float64"
    WASTE_HEAT_MARKET:str= "float64"
    WASTE_HEAT_SELF:str= "float64"
    OTHER_MARKET:str= "float64"
    OTHER_SELF:str= "float64"
    LOAD:str= "float64"

In [8]:
import pandas as pd
import requests
from tqdm import tqdm
from io import StringIO

setup = Setup()
GMH = generation_mix_historical()

schema = GMH.__dict__


years = [2018,2019,2020,2021,2022,2023,2024]
records = list()

df_main = pd.DataFrame()

pbar = tqdm(years[:],leave=True)

for year in pbar:

    pbar.set_description(str(year))

    url = f'https://marketplace.spp.org/file-browser-api/download/generation-mix-historical?path=%2FGenMix_{year}.csv'

    response = requests.get(url)


    buffer = StringIO(response.text)

    if buffer.getvalue():

        df = pd.read_csv(buffer,converters=schema,parse_dates=setup.PARSE_DATES)

        pbar.set_description(str(year) + " " + str(len(response.text)))

        df_main = pd.concat([df_main,df])

    else:
        continue


    

2024: 100%|██████████| 7/7 [01:24<00:00, 12.09s/it]         


In [11]:
s3.write_pyarrow_to_s3(bucket_name="usagrid",object_key="landing_area/SWPP/SWPP_generation_load.arrow",data=df_main)